# Evaluation v2

In [11]:
from picturedata.generator import DataGenerator
from keras.models import Model
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector, merge, Activation, Flatten, GRU, Input
from keras.callbacks import ModelCheckpoint
import numpy as np
import pandas as pd

In [2]:
generator = DataGenerator()

Total samples : 4745052
Vocabulary size: 44537
Maximum caption length: 51


In [3]:
embedding_dim = 128
image_input = Input(shape=(1000,))
image_model = Dense(embedding_dim, input_dim=1000, activation='relu')(image_input)

image_model = RepeatVector(generator.getMaxCapLen())(image_model)

language_input = Input(shape=(generator.getMaxCapLen(),))
language_model = Embedding(generator.getVocabSize(), 256, input_length=generator.getMaxCapLen())(language_input)
language_model = LSTM(256, return_sequences=True)(language_model)
#language_model.add(GRU(128, return_sequences=True))
language_model = TimeDistributed(Dense(embedding_dim))(language_model)

output = merge([image_model, language_model], mode='concat')
output = LSTM(1000, return_sequences=False)(output)
#model.add(GRU(256, return_sequences=False))
output = Dense(generator.getVocabSize())(output)
output = Activation('softmax')(output)

model = Model(inputs=[image_input, language_input], outputs=[output])


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:13: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  del sys.path[0]
/home/ubuntu/.local/lib/python2.7/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [4]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [5]:
model.load_weights('Models/Weights.h5')

In [6]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 51)            0                                            
____________________________________________________________________________________________________
input_1 (InputLayer)             (None, 1000)          0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 51, 256)       11401472    input_2[0][0]                    
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 128)           128128      input_1[0][0]                    
___________________________________________________________________________________________

In [7]:
from pycocotools.coco import COCO

In [8]:
annFile_train = "annotations/captions_train2014.json"

coco = COCO(annFile_train)

loading annotations into memory...
Done (t=1.78s)
creating index...
index created!


In [9]:
imgIds = coco.getImgIds()
annIds = coco.getAnnIds(imgIds)

imgs = coco.loadImgs(imgIds)
anns = coco.loadAnns(annIds)

In [12]:
imgs_df = pd.DataFrame(imgs)
anns_df = pd.DataFrame(anns)

In [13]:
img_new = imgs_df.drop(['coco_url','date_captured','flickr_url','height','width','license'], axis=1)

img_new.rename(columns={'id': 'image_id'}, inplace=True)

anns_df['caption'] = anns_df['caption'].apply(lambda x: '<start>' + x + '<end>')

df = pd.merge(anns_df, img_new, on='image_id', how='left')

In [14]:
df.drop(['id'], axis=1, inplace=True)

In [15]:
df.head()

,caption,image_id,file_name
0,<start>People shopping in an open market for v...,262145,COCO_train2014_000000262145.jpg
1,<start>An open market full of people and piles...,262145,COCO_train2014_000000262145.jpg
2,<start>People are shopping at an open air prod...,262145,COCO_train2014_000000262145.jpg
3,<start>Large piles of carrots and potatoes at ...,262145,COCO_train2014_000000262145.jpg
4,<start>People shop for vegetables like carrots...,262145,COCO_train2014_000000262145.jpg


In [16]:
img_files = set(df['file_name'])